# Cross_Validation and GridSearch

Basic Libraries Import

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Data 불러오기 및 나누기

In [2]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy() # 여러개 할거니까 2차원처럼 입력
target = wine['class'].to_numpy()

# 1. Test_set 대신에 Valid_set 을 만들어 test 하기

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target, test_size = 0.2 ,random_state=42)
sub_input,val_input, sub_target, val_target = train_test_split(train_input,train_target,test_size = 0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


### DecisionTreeClassifier 객체를 생성하여 sub_set 으로 학습하기

In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)

DecisionTreeClassifier(random_state=42)

In [7]:
print(f"Sub_set 의 score : {dt.score(sub_input,sub_target)}")
print(f"validation_set 의 score : {dt.score(val_input,val_target)}")

Sub_set 의 score : 0.9971133028626413
validation_set 의 score : 0.864423076923077


Decision Tree 에 저장된 Parameter Names 를 얻을 수 있음

In [13]:
print(DecisionTreeClassifier().get_params().keys())

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])


# 2. K - fold cross validation
### validation set 을 바꿔가며 Training

In [8]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)

In [9]:
print(scores) # fit_time, score_time,test_score 반환

{'fit_time': array([0.00573134, 0.00501013, 0.00554657, 0.00572395, 0.00469446]), 'score_time': array([0.00099921, 0.00099063, 0.00096393, 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


Cross Validation 결과로 나온 score 들의 평균값

In [10]:
print(np.mean(scores['test_score']))

0.855300214703487


cross_validate 는 '자동으로' 훈련세트를 섞어 폴드를 나누지 않는다.
섞도록 만드려면 cross_validate 의 cv 매개변수 값을 지정해주어야함.

In [11]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)

In [12]:
# 훈련세트를 섞으며 다시 계산한 평균 test_score (Validation set으로 평가)
print(np.mean(scores['test_score']))

0.8574181117533719


# 3. GridSearchCV : 하이퍼파라미터 자동 탐색 + 교차 검증
Hyper_Parameter : 일반적인 parameter 과 달리, 사람이 직접 지정해주는 값

In [28]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002,0.0003,0.0004,0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

검증 점수가 가장 높은 모델이 객체 = gs.best_estimator 로 저장되어 있음.

In [15]:
dt = gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


GridSearchCV 로 찾은 최적의 매개변수 ( params 중에 골랐음)

In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


np.argmax => 가장 값이 큰 element 의 index 반환

In [29]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


#  순서 정리
1. 탐색할 매개변수를 지정한다.
2. Train_set 에서 GridSearch 를 수행하여 최상의 mean Validation score 가 나오는 매개변수 조합을 찾는다. 이 조합은 GridSearch 객체에 저장됨.
3. GridSearch 는 이제 validation_set 없이 전체 Train_set 을 가지고 최종 model 을 fit 한다. 이 모델도 best_estimator_ 에 저장된다.

# 좀 더 복잡한 매개변수 조합을 연습해보자.
3개의 매개변수 조합
실제 DecisionTreeClassifier 에 있는 매개변수 이름을 정확히 입력해야함.

In [30]:
params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)}
# 위 매개변수 조합들로 실행할 교차 검증 횟수 : 9 * 15 * 10 = 1350번
# 각 조합마다 default K 값인 5회씩 모델을 훈련해서 만들어내므로, 1350 * 5 = 6750 개의 model 생성

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1) # n_jobs = -1 : 모든 CPU core 을 사용해 계산하라.
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [21]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### GridSearchCV 를 사용하니 매개변수를 일일이 바꿔가며 교차검증을 수행하지 않아도 됨.
### 원하는 매개변수 값을 나열하면 알아서 교차 검증 후 최상의 매개변수 값을 찾아줌.

# 4. Random Search

매개변수 값의 범위를 미리 정해주기 어려울 때, 매개변수를 Sampling 할 수 있는 확률 분포 객체를 만들어줌.

### 1. 먼저 Scipy에서 2개의 확률 분포 class 를 import 해보자.

In [33]:
from scipy.stats import uniform, randint # uniform = 실수값, randint = 정수값

In [35]:
# method 연습해보기
a = randint(0,10)
print(a.rvs(10))
b = np.unique(a.rvs(1000), return_counts=True)
print(b)
c = uniform(0,1)
c.rvs(10)

[2 1 4 1 5 1 8 3 7 8]
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([112,  90,  83,  83, 103, 135, 107,  99,  94,  94], dtype=int64))


array([0.42738707, 0.77684456, 0.76508564, 0.23110005, 0.5342986 ,
       0.00698323, 0.42244189, 0.91420277, 0.93138741, 0.16866807])

### 2. 이번엔 매개변수 하나를 더 추가하여 4 개의 HyperParameter 에 대한 최적의 조합을 탐색한다.

In [36]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),
         'max_depth' : randint(20,50),
         'min_samples_split' : randint(2,25),
         'min_samples_leaf' : randint(1,25) # leaf node 가 되기 위한 최소 Sample 의 개수
         }

난수의 Sampling 횟수는 sklearn 의 RandomizedSearchCV 의 n_iter 매개변수에서 지정한다.

In [44]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter = 100, n_jobs = -1,random_state=42)
gs.fit(train_input, train_target)
print(f"최고의 HyperParameter 값 : {gs.best_params_}")
print(f"최고의 Cross_Validation Score : {np.max(gs.cv_results_['mean_test_score']):.3f}")

최고의 HyperParameter 값 : {'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
최고의 Cross_Validation Score : 0.870


### 3. 결정된 최적의 모델 gs.best_estimator_ 을 가지고 최종적으로 Test_set 의 점수를 평가해보자.

In [45]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
